In [ ]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Computer Vision with CNNs
#
# For this task you will build a classifier for Rock-Paper-Scissors 
# based on the rps dataset.
#
# IMPORTANT: Your final layer should be as shown, do not change the
# provided code, or the tests may fail
#
# IMPORTANT: Images will be tested as 150x150 with 3 bytes of color depth
# So ensure that your input layer is designed accordingly, or the tests
# may fail. 
#
# NOTE THAT THIS IS UNLABELLED DATA. 
# You can use the ImageDataGenerator to automatically label it
# and we have provided some starter code.


# 주어진문제
import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
    urllib.request.urlretrieve(url, 'rps.zip')
    local_zip = 'rps.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('tmp/')
    zip_ref.close()


    TRAINING_DIR = "tmp/rps/"
    training_datagen = ImageDataGenerator(
    # YOUR CODE HERE)



    train_generator = # YOUR CODE HERE


    model = tf.keras.models.Sequential([
                                        


    # YOUR CODE HERE, BUT END WITH A 3 Neuron Dense, activated by softmax
        tf.keras.layers.Dense(3, activation='softmax')
    ])






    return model



# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("TF3-rps.h5")


In [ ]:
# 데이터를 다운로드 받기 
import urllib.request
import zipfile
import tensorflow as tf

url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
urllib.request.urlretrieve(url, 'rps.zip')
local_zip = 'rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./tmp/')
zip_ref.close()

###  train 데이터셋을 8:2로  train:valid로 나누기 
import os 
import glob 
import random 
import shutil 
import numpy as np 
ROOT_DIR = './tmp/'
TRAINING_DIR = "./tmp/rps/"


#폴더 없으면 만드는 함수 
def createfolder(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

#상위폴더 만들기 
createfolder(ROOT_DIR+'rps_train0')
createfolder(ROOT_DIR+'rps_valid0')

folders = ['paper', 'rock', 'scissors']

#하위폴더만들기 
for f in folders: 
    createfolder(ROOT_DIR+'rps_train0/'+f)
    createfolder(ROOT_DIR+'rps_valid0/'+f)

# 각 폴더에 파일 카피 (랜덤하게 뽑은 인덱스를 이용)
def copy_files(folder):
    files = glob.glob(TRAINING_DIR + folder + '/*.*')
    ratio = 0.2 
    rand_index = np.random.choice(len(files), int(len(files)*ratio), replace=False)

    for i, f in enumerate(files):
        fname = os.path.basename(f)
        if i in rand_index:
            print('{} is copying to train folder'.format(fname))
            shutil.copy2(os.path.join(TRAINING_DIR, folder, fname), os.path.join(ROOT_DIR, 'rps_valid0', folder))
        else:
            print('{} is copying to valid folder'.format(fname))
            shutil.copy2(os.path.join(TRAINING_DIR, folder, fname),  os.path.join(ROOT_DIR,'rps_train0', folder))

# 각 폴더마다 순회하며 카피 
for a in folders: 
    copy_files(a)

In [4]:
# 다시풀기 
def solution_model():
    # url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
    # urllib.request.urlretrieve(url, 'rps.zip')
    # local_zip = 'rps.zip'
    # zip_ref = zipfile.ZipFile(local_zip, 'r')
    # zip_ref.extractall('./tmp/')
    # zip_ref.close()

    TRAINING_DIR = "./tmp/rps_train0/"
    VALID_DIR = "./tmp/rps_valid0"
    training_datagen = ImageDataGenerator(
    # YOUR CODE HERE)
      rescale = 1/255., 
      zoom_range = 0.2, 
      width_shift_range = 0.1, 
      height_shift_range = 0.1 
    )

    train_generator =training_datagen.flow_from_directory(
        TRAINING_DIR, 
        target_size=(150,150),
        batch_size=32,
        class_mode = 'categorical'
    )

    valid_datagen = ImageDataGenerator(
        rescale = 1/255.
    )

    valid_generator = valid_datagen.flow_from_directory(
        VALID_DIR, 
        target_size=(150,150),
        batch_size=32,
        class_mode = 'categorical'
    )

    model = tf.keras.models.Sequential([
          Conv2D(32, (3,3), input_shape=(150,150,3), activation='relu'),                             
          MaxPool2D(2,2),
          Conv2D(64, (3,3), activation='relu'),
          MaxPool2D(2,2),
          Conv2D(128, (3,3), activation='relu'),
          MaxPool2D(2,2),
          Flatten(),
          Dropout(0.5), 
          Dense(256, activation='relu'),
          Dense(128, activation='relu'),
    # YOUR CODE HERE, BUT END WITH A 3 Neuron Dense, activated by softmax
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])
    checkpoint_path = 'rps_checkpoint1-1.ckpt'
    checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                              save_weights_only = True, 
                              save_best_only = True, 
                              monitor='val_loss',
                              verbose=1)
    model.fit(train_generator, validation_data=valid_generator, epochs=20, 
            callbacks=[checkpoint]) 
    model.load_weights(checkpoint_path)
    return model



# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("TF3-rps1-1.h5")

Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Epoch 1/20
63/63 [==============================] - 16s 250ms/step - loss: 1.0811 - acc: 0.4489 - val_loss: 0.1480 - val_acc: 0.9702

Epoch 00001: val_loss improved from inf to 0.14796, saving model to rps_checkpoint1-1.ckpt
Epoch 2/20
63/63 [==============================] - 16s 246ms/step - loss: 0.3371 - acc: 0.8793 - val_loss: 0.0244 - val_acc: 0.9940

Epoch 00002: val_loss improved from 0.14796 to 0.02442, saving model to rps_checkpoint1-1.ckpt
Epoch 3/20
63/63 [==============================] - 16s 249ms/step - loss: 0.1506 - acc: 0.9505 - val_loss: 0.0047 - val_acc: 1.0000

Epoch 00003: val_loss improved from 0.02442 to 0.00471, saving model to rps_checkpoint1-1.ckpt
Epoch 4/20
63/63 [==============================] - 16s 248ms/step - loss: 0.0813 - acc: 0.9695 - val_loss: 0.0305 - val_acc: 0.9881

Epoch 00004: val_loss did not improve from 0.00471
Epoch 5/20
63/63 [==============================]